In [ ]:
from pymongo import MongoClient
from pddoc2vec import doc2vec
from tqdm import tqdm
import pandas as pd


client = MongoClient()
client = MongoClient('localhost', 27017)  #Connects to the MongoDB, make sure youre SSH'ed into the docker

collection_list = client['twitter'].collection_names() #Scrapes all the collection names from the Docker

sentimentdata = []
for posts in tqdm(collection_list):
    for post in client['twitter'][posts].find({'sentiment': {"$exists": True}}):  #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

df = pd.DataFrame(sentimentdata) # pulls data into the Pandas DataFrame


print(df.head(10))

/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  # Remove the CWD from sys.path while we load stuff.
 58%|█████▊    | 11/19 [00:32<00:23,  3.00s/it]

In [ ]:
print(df.columns.values) #How many Columns we have in our dataframe


In [ ]:

df['Source'].value_counts() 

#How many sources our data comes from,  different sources might label the 
#same data under different names/labels

In [ ]:
print(df.loc[df['Source'] == 'Misc_MiscFilmReviews_M'].sample(4))

# This source contains the text under "text" 


In [ ]:
print(df.loc[df['Source'] == 'LabeledStatuses_Sandy_K'].sample(4))

# This source contains the text under "TweetBody" and has a "classification"
# field

In [ ]:

df["main"] = df["sentiment"]
# we are going to build a main label that is just the sentiment

df["sentiment"] = df["Source"] + "_" + df["sentiment"]
# We add source informationg with the sentiment

df['text'].fillna(df['TweetBody'], inplace=True)
# we move the text from tweetbody into text

df['classification'] = df['classification'].fillna("0")
#we populate empty classification  fields with "0"


print(df.isna().sum())

df = df.applymap(str)
print(df.isna().sum())



In [ ]:
print("Building model")

# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(df, "text", ["main",  "Source", 'sentiment', 'classification'])


print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
print(x.score())

print("Predicting on text")

# You can predict text as follows
print(x.predict_text("its going to be the worst time ever"))


In [ ]:
# how to save model https://radimrehurek.com/gensim/models/doc2vec.html

from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")
x.model.save(fname)
x.model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

# If you’re finished training a model (=no more updates, only querying, reduce memory usage), you can do:

x.model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)



